In [14]:
!pip install cvxpy
import numpy as np
import pandas as pd
from itertools import combinations
from itertools import permutations
from itertools import product
from scipy.optimize import linprog
import cvxpy as cp

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


Set of alternatives $X$ where $|X| = n < \infty$. Set of all non-empty menus on $X$ denoted $\mathcal{A}$.<br><br>
 We assume that choice proportions of all alternatives from every menu are observed. $\rho_A(a)$ represents the probability that alternative $a$ is chosen from menu $A$.<br><br>
Naturally, $\rho_A(b) = 0$ if $b\not\in A$,
$$
\sum_{a\in A}\rho_A(a) = 1
$$
and $\rho_{A}(a) \in [0,1]$ for all $a, A$ pairs.
<br><br>
Let the set of all linear orders (complete, transitive and asymmetric binary relations) on $X$ be denoted as $\Pi$. We say that a dataset $\rho$ is random utility rationalizable if there exists some probability measure $\mu \in \Delta(\Pi)$ such that, for all $A\subseteq X\backslash \emptyset$ and $a\in A$, we have
$$
\rho_A(a) = \mu(\{\pi \in \Pi: a \succeq_\pi b \ \forall \ b\in A\backslash \{a\}\})
$$
<br>
In words, the probability that any element is chosen from a menu must be equal to the probability of drawing a linear order according to which that alternative is the most preferred from that menu. 
<br><br>
This is equivalent to $\mu$ being a probability vector, and solution to the equation

$$
C\cdot \mu = \rho
$$

Where $C$ is the matrix of choice functions on $2^X$. I.e if $X = \{x,y\}$ then there are two linear orders, either $x\succ y$ or $y\succ x$. Additionally, there is only one (non-trivial) menu: $\{x,y\}$. The matrix $C$ would therefore take the form:


| | $x\succ y$ | $y\succ x$ |
| --- | --- | --- |
| $x, \{x,y\}$ | 1 | 0 |
| $y, \{x,y\}$ | 0 | 1 |


For example, if we had $\rho = \langle 0.4, 0.6 \rangle$ then clearly the solution to the equation $C\cdot \mu = \rho$ would be $\mu = \langle 0.4, 0.6\rangle$.
<br><br>
When $n \geq 3$ there is no guarantee that there exists a solution. So, instead of looking for a distribution over the support of linear orders (or, equivalently, the set of rational choice functions), let's consider a distribution over the set of irrational choice functions also. This increases the number of free parameters and therefore increases the likelihood that one can find a solution. 
<br><br>
Given that 'distance' over the space of linear orders can take many forms, and that a preferred one is typically not aggreed upon, we will define the matrix $C_i^\pi$ as a matrix in which each column provides the choices for each menu of a choice function that agrees with the choice function according to $\pi$ over all menus besides $i$ of them. 
<br><br>
For example, suppose $X = \{x,y,z\}$ and $\pi = x\succ y \succ z$. The column of the matrix $C$ corresponding to this linear order would look like

| alt, menu | chosen |
| --- | --- |
| $x, \{x,y\}$| $1$ |
| $y, \{x,y\}$| $0$ |
| $x, \{x,z\}$| $1$ |
| $z, \{x,z\}$| $0$ |
| $y, \{y,z\}$| $1$ |
| $z, \{y,z\}$| $0$ |
| $x, \{x,y,z\}$| $1$ |
| $y, \{x,y,z\}$| $0$ |
| $z, \{x,y,z\}$| $0$ |


Then the matrix $C_1^\pi$ would look like:

| alt, menu | original | | | | | |
| --- | --- | --- | --- | --- | --- | --- |
| $x, \{x,y\}$| $1$ | $0$ | $1$ | $1$ | $1$ | $1$ |
| $y, \{x,y\}$| $0$ | $1$ | $0$ | $0$ | $0$ | $0$ |
| $x, \{x,z\}$| $1$ | $1$ | $0$ | $1$ | $1$ | $1$ |
| $z, \{x,z\}$| $0$ | $0$ | $1$ | $0$ | $0$ | $0$ |
| $y, \{y,z\}$| $1$ | $1$ | $1$ | $0$ | $1$ | $1$ |
| $z, \{y,z\}$| $0$ | $0$ | $0$ | $1$ | $0$ | $0$ |
| $x, \{x,y,z\}$| $1$ | $1$ | $1$ | $1$ | $0$ | $0$ |
| $y, \{x,y,z\}$| $0$ | $0$ | $0$ | $0$ | $1$ | $0$ |
| $z, \{x,y,z\}$| $0$ | $0$ | $0$ | $0$ | $0$ | $1$ |


Let $C_i$ represent $\cup_{\pi \in \Pi} C_i^\pi$. And if $R = \cup_{i = 1}^{n}$ then we are guaranteed to have a solution to the following system $R \mu = \rho$.
<br><br>
However, this is not particularly interesting, as we would like to see how much weight, if possible, we can attribute to rational decision makers. Beginning with $i = 1$, consider the following problem.
<br>
Assume $C$ is a $n\times m$ matrix and $C_1$ is a $n\times k$ matrix. This means that we have $n+k$ values in our vector $\mu$ and $n+k-1$ free parameters. We wish to maximize the weight placed on the rational choice functions and so we can minimize the following function
$$
\min_{\mu} -\sum_{i=1}^n w_i \mu_i + \sum_{j=n+1}^{n+k} w_j \mu_j \\
\\
\\
s.t. \ \ \ \tilde{C}\mu = \rho \\
\\
\sum_{i=1}^{n+k}\mu_i = 1 \\
\\
\mu_i \geq 0
$$
Where $\tilde{C} = [C, C_i]$, $w_i$ is large and positive for the first $n$ and $w_i$ is small and positive for the last $k$. 
<br>
<br>
This allows us to fit a random utility representation over the whole set of rational and level 1 irrational choice functions while maximizing the weight on the rational decision makers.


In [15]:
class ChoiceModel:
    def __init__(self, n):
        self.n = n
        self.X = list(range(1, n+1))
        self.probs = self.simulate_probs()
        self.rows = self.probs.index
        self.columns = list(permutations(self.X))
        self.C = self.create_C()
        self.RationalRUM = self.trad_RUM()

    def simulate_probs(self, alpha=1.0):
        menus = []
        for r in range(2, len(self.X)+1):
            menus.extend(combinations(self.X, r))
        menu_probs = {}
        for menu in menus: 
            k = len(menu)
            probs = np.random.dirichlet([alpha]*k)
            menu_probs[menu] = dict(zip(menu, probs))

        data = []
        index = []

        for menu, probs in menu_probs.items():
            for alt, p in probs.items():
                index.append((alt, menu))
                data.append(p)
        idx = pd.MultiIndex.from_tuples(index, names=['alternative', 'menu'])

        return pd.Series(data, index=idx, name='choice_probability').sort_index(level='menu')
        
    def create_C(self):
        idx = []
        for alternative, menu in self.rows:
            for ranking in self.columns:
                rank_int = tuple(a for a in ranking if a in menu)
                if rank_int.index(alternative) == 0:
                    idx.append((alternative, menu, ranking, 1))
                else:
                    idx.append((alternative, menu, ranking, 0))
        alt_main = [item[0] for item in idx]
        menu_main = [item[1] for item in idx]
        ranking_main = [item[2] for item in idx]
        choice_main = [item[3] for item in idx]
        C = pd.DataFrame({'alternative': alt_main, 'menu': menu_main, 'ranking': ranking_main, 'choice': choice_main})
        C = C.pivot(index=['alternative', 'menu'], columns='ranking', values='choice')
        C = C.sort_index(level ='menu')

        return C
    
    def trad_RUM(self):
        C = self.C.to_numpy()
        p = self.probs.to_numpy().reshape(-1, 1)

        n = C.shape[1]
        c = np.zeros(n)
        bounds = [[0,1] for _ in range(n)]
        C_eq = np.vstack([C, np.ones(n)])
        p_eq = np.append(p, 1)

        res = linprog(c, A_eq=C_eq, b_eq=p_eq, bounds=bounds, method='highs')
        
        return res.success
    
    def generate_mistake_functions(self, df, col, d):
        menus = df.index.get_level_values("menu").unique()

        true_choice = {
            m: df[col].xs(m, level="menu").idxmax()
            for m in menus
        }

        mistake_alts = {
            m: [a for a in df[col].xs(m, level="menu").index if a != true_choice[m]]
            for m in menus
        }

        all_cf = []  

        for mistaken_menus in combinations(menus, d):

            wrong_lists = [mistake_alts[m] for m in mistaken_menus]

            for wrong_combo in product(*wrong_lists):

                cf = {}
                wrong_map = dict(zip(mistaken_menus, wrong_combo))

                for m in menus:
                    if m in wrong_map:
                        cf[m] = wrong_map[m]         
                    else:
                        cf[m] = true_choice[m]       

                all_cf.append(cf)

        return all_cf

    def mistake_df(self, df, col, d):

        mistake_functions = self.generate_mistake_functions(df, col, d)

        new_cols = []
        new_data = []

        for i, cf in enumerate(mistake_functions):
            col_name = f"m_{str(col)}_{d}_{i}"  
            new_cols.append(col_name)

            col_values = []

            for (alt, menu) in df.index:    
                chosen = cf[menu]           
                col_values.append(1 if alt == chosen else 0)

            new_data.append(col_values)

        new_df = pd.DataFrame(
            np.column_stack(new_data),
            index=df.index,
            columns=new_cols
        )

        return new_df
    
    def create_Ci(self, d=1):
        C = self.C
        main = pd.DataFrame()
        irr = pd.DataFrame()
        for i in range(1, d+1):
            for ranking in C.columns:
                irr = pd.concat([irr, self.mistake_df(C, ranking, i)], axis=1)
            main = pd.concat([main, irr], axis=1)
        nCols = [int(col.split("_")[2]) for col in main.columns]
        return main, nCols

    def solve(self, depth=0):
        if depth==0:
            s = self.trad_RUM()
            info = None
        else:    
            s = False
            C = self.C
            
            C_i, levels = self.create_Ci(d=depth)
            rho = self.probs
            m, n = C.shape
            _, k = C_i.shape

            allLevels = [0]*n + levels
            allLevels = [100**(1/(l+1)) for l in allLevels]

            main = pd.concat([C, C_i], axis=1)

            w = -1*np.array(allLevels)

            A_eq = main.to_numpy()
            b_eq = rho.to_numpy()
            A_eq_sum = np.ones((1, n+k))
            b_eq_sum = np.array([1])

            A_eq_total = np.vstack([A_eq, A_eq_sum])
            b_eq_total = np.concatenate([b_eq, b_eq_sum])

            bounds = [(0,1)]*(n+k)

            result = linprog(w, A_eq = A_eq_total, b_eq = b_eq_total, bounds=bounds, method='highs')
            s = result.success
            if result.success:
                r = pd.DataFrame({'lo': [item1 for item1, _ in zip(list(main.columns), list(result.x))], 'weight': [float(item2) for _, item2 in zip(list(main.columns), list(result.x))]})
                #r['exp_level'] = r['lo'].apply(lambda x: 0 if x in C.columns else levels[x])
                info = r
            else:
                info = None

        return s, info
    
    def iterate(self):
        depth = -1
        success = False
        while not success:
            depth += 1
            success, data = self.solve(depth=depth)
            
        return data

In [16]:
test = ChoiceModel(4)
df = test.iterate()

In [18]:
df[df['weight'] > 0]
df['linearOrder']= df['lo'].apply(lambda x: x if isinstance(x, tuple) else x.split("_")[1])
df['rational'] = df['lo'].apply(lambda x: isinstance(x, tuple))
df['depth'] = df['lo'].apply(lambda x: 0 if isinstance(x, tuple) else int(x.split("_")[2]))
df[df['weight']>0].groupby(['linearOrder', 'rational'])[['weight', 'depth']].agg({'weight': 'sum', 'depth':'mean'})

,,weight,depth
linearOrder,rational,,
"(3, 1, 2, 4)",True,0.041751,0.0
"(4, 2, 3, 1)",True,0.042115,0.0
"(1, 2, 3, 4)",False,0.249882,2.0
"(2, 1, 3, 4)",False,0.184125,2.0
"(2, 3, 1, 4)",False,0.104828,2.0
"(3, 1, 2, 4)",False,0.103993,2.0
"(4, 2, 3, 1)",False,0.273307,2.0


In [29]:
def data_gen(n):
    alternatives = list(range(n))
    linear_orders = list(permutations(alternatives, n))
    information = pd.DataFrame({'lo': linear_orders})
    for i in range(n):
        information[f"u{i}"] = information['lo'].apply(lambda x: 10*x.index(i))
    information['lo'] = information['lo'].apply(lambda x: x[::-1])

    return information

In [30]:
data_gen(4)

,lo,u0,u1,u2,u3
0,"(3, 2, 1, 0)",0,10,20,30
1,"(2, 3, 1, 0)",0,10,30,20
2,"(3, 1, 2, 0)",0,20,10,30
3,"(1, 3, 2, 0)",0,30,10,20
4,"(2, 1, 3, 0)",0,20,30,10
5,"(1, 2, 3, 0)",0,30,20,10
6,"(3, 2, 0, 1)",10,0,20,30
7,"(2, 3, 0, 1)",10,0,30,20
8,"(3, 0, 2, 1)",20,0,10,30
9,"(0, 3, 2, 1)",30,0,10,20
